## Part 1: Preprocessing

In [27]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras import layers

#  Import and read the attrition data
attrition_df = pd.read_csv('https://static.bc-edx.com/ai/ail-v-1-0/m19/lms/datasets/attrition.csv')
attrition_df.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,HourlyRate,JobInvolvement,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,Sales,1,2,Life Sciences,2,94,3,...,3,1,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,Research & Development,8,1,Life Sciences,3,61,2,...,4,4,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,Research & Development,2,2,Other,4,92,2,...,3,2,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,Research & Development,3,4,Life Sciences,4,56,3,...,3,3,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,Research & Development,2,1,Medical,1,40,3,...,3,4,1,6,3,3,2,2,2,2


In [2]:
# Determine the number of unique values in each column.
attrition_df.nunique()

,0
Age,43
Attrition,2
BusinessTravel,3
Department,3
DistanceFromHome,29
Education,5
EducationField,6
EnvironmentSatisfaction,4
HourlyRate,71
JobInvolvement,4


In [4]:
# Create y_df with the Attrition and Department columns
y_df = attrition_df[['Attrition', 'Department']]


In [9]:
# Create a list of at least 10 column names to use as X data
X_columns = ['Age', 'DistanceFromHome', 'Education', 'EnvironmentSatisfaction',
             'JobInvolvement', 'JobLevel', 'JobSatisfaction', 'YearsInCurrentRole', 'NumCompaniesWorked', 'TotalWorkingYears']


# Create X_df using your selected columns
X_df = attrition_df[X_columns]

# Show the data types for X_df
print(X_df.dtypes)


Age                        int64
DistanceFromHome           int64
Education                  int64
EnvironmentSatisfaction    int64
JobInvolvement             int64
JobLevel                   int64
JobSatisfaction            int64
YearsInCurrentRole         int64
NumCompaniesWorked         int64
TotalWorkingYears          int64
dtype: object


In [11]:
# Split the data into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, test_size=0.2, random_state=42)


In [ ]:
# Convert your X data to numeric data types however you see fit
# Add new code cells as necessary


No     788
Yes    314
Name: OverTime, dtype: int64

In [17]:
# Create a StandardScaler
scaler = StandardScaler()


# Fit the StandardScaler to the training data
X_train_scaled = scaler.fit_transform(X_train)

# Scale the training and testing data
X_test_scaled = scaler.transform(X_test)


In [20]:
# Create a OneHotEncoder for the Department column
encoder_dep = OneHotEncoder(sparse=False)

# Fit the encoder to the training data
y_train_dep_encoded = encoder_dep.fit_transform(y_train[['Department']])

# Create two new variables by applying the encoder
# to the training and testing data
y_test_dep_encoded = encoder_dep.transform(y_test[['Department']])



/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [26]:
# Create a OneHotEncoder for the Attrition column
encoder_attr = OneHotEncoder(sparse=False)

# Fit the encoder to the training data
y_train_attr_encoded = encoder_attr.fit_transform(y_train[['Attrition']])

# Create two new variables by applying the encoder
# to the training and testing data
y_test_attr_encoded = encoder_attr.transform(y_test[['Attrition']])

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


## Create, Compile, and Train the Model

In [28]:
# Find the number of columns in the X training data
input_dim = X_train_scaled.shape[1]

# Create the input layer
input_layer = Input(shape=(input_dim,))

# Create at least two shared layers
shared_layer_1 = Dense(64, activation='relu')(input_layer)
shared_layer_2 = Dense(32, activation='relu')(shared_layer_1)

In [32]:
# Create a branch for Department
# with a hidden layer and an output layer

# Create the hidden layer
deptartment_branch = Dense(16, activation='relu')(shared_layer_2)


# Create the output layer
deptartment_output = Dense(y_train_dep_encoded.shape[1], activation='softmax', name='department_output')(deptartment_branch)



In [31]:
# Create a branch for Attrition
# with a hidden layer and an output layer

# Create the hidden layer
attrition_branch = Dense(16, activation='relu')(shared_layer_2)

# Create the output layer
attrition_output = Dense(y_train_attr_encoded.shape[1], activation='sigmoid', name='attrition_output')(attrition_branch)



In [33]:
# Create the model
model = Model(inputs=input_layer, outputs=[deptartment_output, attrition_output])

# Compile the model
model.compile(optimizer='adam',
              loss={'department_output': 'categorical_crossentropy', 'attrition_output': 'binary_crossentropy'},
              metrics={'department_output': 'accuracy', 'attrition_output': 'accuracy'})

# Summarize the model
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 10)]                 0         []                            
                                                                                                  
 dense (Dense)               (None, 64)                   704       ['input_1[0][0]']             
                                                                                                  
 dense_1 (Dense)             (None, 32)                   2080      ['dense[0][0]']               
                                                                                                  
 dense_5 (Dense)             (None, 16)                   528       ['dense_1[0][0]']             
                                                                                              

In [34]:
# Train the model
history = model.fit(X_train_scaled,
                    {'department_output': y_train_dep_encoded, 'attrition_output': y_train_attr_encoded},
                    epochs=100,
                    batch_size=32,
                    validation_split=0.2)


Epoch 1/100
30/30 [==============================] - 2s 16ms/step - loss: 1.5888 - department_output_loss: 0.9842 - attrition_output_loss: 0.6046 - department_output_accuracy: 0.5138 - attrition_output_accuracy: 0.8255 - val_loss: 1.3809 - val_department_output_loss: 0.8223 - val_attrition_output_loss: 0.5585 - val_department_output_accuracy: 0.6314 - val_attrition_output_accuracy: 0.7966
Epoch 2/100
30/30 [==============================] - 0s 4ms/step - loss: 1.2995 - department_output_loss: 0.8104 - attrition_output_loss: 0.4891 - department_output_accuracy: 0.6543 - attrition_output_accuracy: 0.8404 - val_loss: 1.2927 - val_department_output_loss: 0.7774 - val_attrition_output_loss: 0.5153 - val_department_output_accuracy: 0.6314 - val_attrition_output_accuracy: 0.7966
Epoch 3/100
30/30 [==============================] - 0s 4ms/step - loss: 1.2196 - department_output_loss: 0.7771 - attrition_output_loss: 0.4424 - department_output_accuracy: 0.6553 - attrition_output_accuracy: 0.8404

In [35]:
# Evaluate the model with the testing data
loss, dept_loss, attr_loss, dept_accuracy, attr_accuracy = model.evaluate(X_test_scaled,
                                                                         {'department_output': y_test_dep_encoded, 'attrition_output': y_test_attr_encoded})

10/10 [==============================] - 0s 4ms/step - loss: 1.9503 - department_output_loss: 1.4326 - attrition_output_loss: 0.5177 - department_output_accuracy: 0.6224 - attrition_output_accuracy: 0.8503


In [36]:
# Print the accuracy for both department and attrition
print(f"Department predictions accuracy: {dept_accuracy}")
print(f"Attrition predictions accuracy: {attr_accuracy}")

Department predictions accuracy: 0.6224489808082581
Attrition predictions accuracy: 0.8503401279449463


# Summary

In the provided space below, briefly answer the following questions.

1. Is accuracy the best metric to use on this data? Why or why not?

Accuracy is okay, but if most employees stay and only a few leave, the model might just predict "stay" all the time and still look accurate. Metrics like precision, recall, and F1-score can give a clearer picture of how well the model handles employees who leave.

2. What activation functions did you choose for your output layers, and why?

For predicting the department, I used softmax, which gives a probability for each department. For predicting if someone will leave, I used sigmoid, which gives a probability between 0 and 1.

3. Can you name a few ways that this model might be improved?

We could adjust the number of layers and learning rate, add or refine features, balance the data if there's an imbalance, use dropout layers to prevent overfitting, apply cross-validation for reliability, and try advanced models like recurrent neural networks.

YOUR ANSWERS HERE

1.
2.
3.